In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# plt.rcParams['font.family']='Times New Roman,Microsoft YaHei'# 设置字体族，中文为微软雅黑，英文为Times New Roman
plt.rcParams['font.sans-serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 设置数学公式字体为stix
plt.rcParams["text.usetex"] = False
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import requests
import xarray as xr
import os
from netCDF4 import Dataset
from openaq import OpenAQ
import time
from meteostat import Point, Hourly,Daily
from tqdm import tqdm
from datetime import datetime, timedelta
import logging


In [2]:
A = np.array([[1, 1, 1, 0, 2], 
              [1, 0, 1, 1, 3], 
              [0, 1, 1, 1, 4]])

U, S, VT = np.linalg.svd(A)
print("U:\n", U)
print("S:", S)
print("VT:\n", VT)

NameError: name 'np' is not defined

In [ ]:
U, S, VT = np.linalg.svd(A)

# 创建 Sigma+
S_plus = np.zeros((A.shape[1], A.shape[0]))
for i in range(len(S)):
    if S[i] > 1e-10:  # 避免除以零
        S_plus[i, i] = 1 / S[i]

# 计算广义逆
A_plus = VT.T @ S_plus @ U.T
print("Moore-Penrose 广义逆 A+:\n", A_plus)


In [ ]:
dataiso = pd.read_pickle('isoprene_data.pkl')
datatempe = pd.read_pickle('temperature_data.pkl')

In [ ]:
# 将'Date'列转换为日期时间格式
dataiso['Date'] = pd.to_datetime(dataiso['Date'])
datatempe['Date'] = pd.to_datetime(datatempe['Date'])

# 确保latitude和longitude为浮点数类型
dataiso['latitude'] = pd.to_numeric(dataiso['latitude'], errors='coerce')
dataiso['longitude'] = pd.to_numeric(dataiso['longitude'], errors='coerce')
datatempe['latitude'] = pd.to_numeric(datatempe['latitude'], errors='coerce')
datatempe['longitude'] = pd.to_numeric(datatempe['longitude'], errors='coerce')

# 解析'time_resolution'为Timedelta类型
def parse_time_resolution(res):
    try:
        delta = pd.to_timedelta(res)
    except Exception:
        if res.endswith('s'):
            seconds = int(res[:-1])
            delta = pd.Timedelta(seconds=seconds)
        elif res.endswith('min'):
            minutes = int(res[:-3])
            delta = pd.Timedelta(minutes=minutes)
        elif res.endswith('h'):
            hours = int(res[:-1])
            delta = pd.Timedelta(hours=hours)
        elif res.endswith('d'):
            days = int(res[:-1])
            delta = pd.Timedelta(days=days)
        else:
            delta = pd.NaT
    return delta

# 计算每条记录的时间区间
dataiso['time_resolution_timedelta'] = dataiso['time_resolution'].apply(parse_time_resolution)
datatempe['time_resolution_timedelta'] = datatempe['time_resolution'].apply(parse_time_resolution)

dataiso['start_time'] = dataiso['Date']
dataiso['end_time'] = dataiso['Date'] + dataiso['time_resolution_timedelta']
datatempe['start_time'] = datatempe['Date']
datatempe['end_time'] = datatempe['Date'] + datatempe['time_resolution_timedelta']

In [ ]:
# 初始化一个空的DataFrame来存储结果
matched_data = pd.DataFrame()

# 获取所有的station_id
station_ids = dataiso['station_id'].unique()

# 对每个station_id进行处理，添加进度条
for station_id in tqdm(station_ids, desc='Processing stations'):
    # 获取相同station_id的dataiso和datatempe数据
    iso_subset = dataiso[dataiso['station_id'] == station_id].copy()
    temp_subset = datatempe[datatempe['station_id'] == station_id].copy()
    
    # 按照时间排序，便于后续匹配
    iso_subset = iso_subset.sort_values('start_time').reset_index(drop=True)
    temp_subset = temp_subset.sort_values('start_time').reset_index(drop=True)
    
    # 如果有温度数据，进行匹配
    if not temp_subset.empty:
        # 为温度数据创建时间索引
        temp_subset.set_index(['start_time', 'end_time'], inplace=True)
        
        # 定义函数，匹配每个异戊二烯记录的温度数据
        def match_temperature(row):
            # 找到时间区间有重叠的温度数据
            overlap = temp_subset[
                (temp_subset.index.get_level_values('start_time') <= row['end_time']) & 
                (temp_subset.index.get_level_values('end_time') >= row['start_time'])
            ]
            if not overlap.empty:
                # 计算匹配到的温度数据的平均值
                avg_temp = overlap['temperature'].mean()
                return avg_temp
            else:
                return np.nan
        
        # 应用匹配函数
        iso_subset['temperature'] = iso_subset.apply(match_temperature, axis=1)
    else:
        # 如果没有温度数据，温度列设为NaN
        iso_subset['temperature'] = np.nan
    
    # 将结果添加到matched_data中
    matched_data = pd.concat([matched_data, iso_subset], ignore_index=True)

# 找出没有匹配到温度数据的记录
missing_temps = matched_data[matched_data['temperature'].isna()].copy()

# 对缺失温度数据的站点进行分组处理
missing_groups = missing_temps.groupby('station_id')

# 使用meteostat获取缺失的温度数据，并考虑时间分辨率
for station_id, group in tqdm(missing_groups, desc='Fetching Meteostat data'):
    lat = group['latitude'].iloc[0]
    lon = group['longitude'].iloc[0]
    
    # 获取该站点所有缺失数据的时间范围
    start_time = group['start_time'].min()
    end_time = group['end_time'].max()

    # 创建meteostat的Point对象
    location = Point(lat, lon)
    
    # 获取小时级别的温度数据
    try:
        data = Hourly(location, start_time, end_time)
        data = data.fetch()
    except Exception as e:
        print(f"Error fetching Meteostat data for station {station_id}: {e}")
        continue

    if not data.empty:
        # 将时间列转换为datetime，并设置为索引
        data.index = pd.to_datetime(data.index)
        data.sort_index(inplace=True)
    else:
        continue

    # 对于group中的每一行，按照时间分辨率匹配温度数据
    for idx, row in group.iterrows():
        # 获取当前记录的时间分辨率
        res = row['time_resolution_timedelta']
        # 获取当前记录的时间范围
        st = row['start_time']
        et = row['end_time']
        
        # 如果分辨率小于1小时，需要对温度数据进行插值
        if res < pd.Timedelta(hours=1):
            # 创建包含温度数据和时间索引的Series
            temp_series = data['temp']
            # 创建插值时间索引
            interp_index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='1min')
            # 使用线性插值
            temp_interp = temp_series.reindex(interp_index)
            temp_interp = temp_interp.interpolate(method='time')
            # 在当前记录的时间范围内取平均温度
            mask = (temp_interp.index >= st) & (temp_interp.index <= et)
            avg_temp = temp_interp.loc[mask].mean()
            matched_data.loc[idx, 'temperature'] = avg_temp
        else:
            # 对于1小时或以上的分辨率，直接使用温度数据
            mask = (data.index >= st) & (data.index <= et)
            relevant_data = data.loc[mask]
            if not relevant_data.empty:
                avg_temp = relevant_data['temp'].mean()
                matched_data.loc[idx, 'temperature'] = avg_temp
                 

# 检查是否还有未匹配到的温度数据
unmatched = matched_data[matched_data['temperature'].isna()]
print(f"Number of unmatched records: {len(unmatched)}")

# 对于仍未匹配到的，可以选择用平均值填充，或者保持为NaN
# matched_data['temperature'].fillna(matched_data['temperature'].mean(), inplace=True)

# # 选择需要保留的列
# columns_to_keep = [
#     'Date', 'isoprene_ng_per_m3_amean', 'temperature', 'longitude', 'latitude',
#     'monitoring_equipment', 'station_name', 'station_id', 'time_resolution'
# ]
# matched_data = matched_data[columns_to_keep]

# 重命名列名，使其更清晰
matched_data = matched_data.rename(columns={
    'isoprene_ng_per_m3_amean': 'isoprene_amean'
})


In [ ]:
# 湿度处理(懒得改变量名)
missing_groups = matched_data.groupby('station_id')

# 使用meteostat获取缺失的温度数据，并考虑时间分辨率
for station_id, group in tqdm(missing_groups, desc='Fetching Meteostat data'):
    lat = group['latitude'].iloc[0]
    lon = group['longitude'].iloc[0]
    
    # 获取该站点所有缺失数据的时间范围
    start_time = group['start_time'].min()
    end_time = group['end_time'].max()

    # 创建meteostat的Point对象
    location = Point(lat, lon)
    
    # 获取小时级别的温度数据
    try:
        data = Hourly(location, start_time, end_time)
        data = data.fetch()
    except Exception as e:
        print(f"Error fetching Meteostat data for station {station_id}: {e}")
        continue

    if not data.empty:
        # 将时间列转换为datetime，并设置为索引
        data.index = pd.to_datetime(data.index)
        data.sort_index(inplace=True)
    else:
        continue

    # 对于group中的每一行，按照时间分辨率匹配温度数据
    for idx, row in group.iterrows():
        # 获取当前记录的时间分辨率
        res = row['time_resolution_timedelta']
        # 获取当前记录的时间范围
        st = row['start_time']
        et = row['end_time']
        
        # 如果分辨率小于1小时，需要对温度数据进行插值
        if res < pd.Timedelta(hours=1):
            # 创建包含温度数据和时间索引的Series
            temp_series = data['rhum']
            # 创建插值时间索引
            interp_index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='1min')
            # 使用线性插值
            temp_interp = temp_series.reindex(interp_index)
            temp_interp = temp_interp.interpolate(method='time')
            # 在当前记录的时间范围内取平均温度
            mask = (temp_interp.index >= st) & (temp_interp.index <= et)
            avg_temp = temp_interp.loc[mask].mean()
            matched_data.loc[idx, 'humidity'] = avg_temp
        else:
            # 对于1小时或以上的分辨率，直接使用温度数据
            mask = (data.index >= st) & (data.index <= et)
            relevant_data = data.loc[mask]
            if not relevant_data.empty:
                avg_temp = relevant_data['rhum'].mean()
                matched_data.loc[idx, 'humidity'] = avg_temp

In [ ]:
station_ids = dataiso['station_id'].unique()

# NASA POWER API URL
nasa_api_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"

# NASA 参数配置
nasa_params = {
    'community': 'AG',
    'parameters': 'ALLSKY_SFC_SW_DWN,ALLSKY_SFC_LW_DWN',  # 包含短波和长波辐射数据
    'format': 'JSON',
}

def fetch_nasa_radiation(lat, lon, start_date, end_date):
    # 构建 NASA API 请求参数
    params = nasa_params.copy()
    params.update({
        'latitude': lat,
        'longitude': lon,
        'start': start_date.strftime('%Y%m%d'),
        'end': end_date.strftime('%Y%m%d'),
    })
    try:
        response = requests.get(nasa_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if 'properties' in data and 'parameter' in data['properties']:
            # 提取短波和长波辐射数据并计算净辐射
            sw_radiation = data['properties']['parameter']['ALLSKY_SFC_SW_DWN']
            lw_radiation = data['properties']['parameter']['ALLSKY_SFC_LW_DWN']
            net_radiation_data = {k: sw_radiation[k] + lw_radiation[k] for k in sw_radiation.keys()}
            radiation_series = pd.Series(net_radiation_data)
            radiation_series.index = pd.to_datetime(radiation_series.index, format='%Y%m%d%H')
            return radiation_series
    except Exception as e:
        print(f"Error fetching NASA radiation data: {e}")
        return pd.Series(dtype=float)

# 对 `matched_data` 中每个站点处理辐射数据
for station_id, group in tqdm(matched_data.groupby('station_id'), desc='Processing stations'):
    lat = group['latitude'].iloc[0]
    lon = group['longitude'].iloc[0]
    start_time = group['start_time'].min()
    end_time = group['end_time'].max()

    # 获取该站点的 NASA 辐射数据
    radiation_data = fetch_nasa_radiation(lat, lon, start_time, end_time)

    # 逐行处理 matched_data 中的每一条记录，并根据时间分辨率进行插值或平均
    for idx, row in group.iterrows():
        res = row['time_resolution_timedelta']
        st, et = row['start_time'], row['end_time']
        
        if not radiation_data.empty:
            if res < pd.Timedelta(hours=1):  # 分辨率小于1小时，插值
                interp_index = pd.date_range(radiation_data.index.min(), radiation_data.index.max(), freq='1min')
                radiation_interp = radiation_data.reindex(interp_index).interpolate(method='time')
                mask = (radiation_interp.index >= st) & (radiation_interp.index <= et)
                avg_radiation = radiation_interp.loc[mask].mean()
            else:  # 分辨率等于或大于1小时，直接平均
                mask = (radiation_data.index >= st) & (radiation_data.index <= et)
                avg_radiation = radiation_data.loc[mask].mean()

            # 将净辐射数据添加到 matched_data 中
            matched_data.loc[idx, 'ground_radiation'] = avg_radiation
        else:
            matched_data.loc[idx, 'ground_radiation'] = np.nan


In [ ]:
# matched_data.to_pickle('matched_data.pkl')
matched_data=pd.read_pickle('matched_data.pkl')

In [ ]:
# 初始化 OpenAQ 客户端，传递您的 OpenAQ API Key
client = OpenAQ(api_key="07852d11a2dd76ad8f7c78e54ad9c972d967790c961b1eecb0a0774a4c0cd0c1")

# 定义所需污染物参数
pollutants = ['nox','pm25', 'pm10', 'so2', 'no2', 'o3']

In [ ]:
# 配置日志记录
logging.basicConfig(
    filename='script_log.txt',          # 日志文件名
    filemode='w',                       # 写入模式，'w'表示覆盖，'a'表示追加
    level=logging.INFO,                 # 日志级别
    format='%(asctime)s - %(levelname)s - %(message)s',  # 日志格式
    datefmt='%Y-%m-%d %H:%M:%S'         # 日期格式
)

In [12]:
match

,Date,isoprene_amean,isoprene_ng_per_m3_amean_qc,isoprene_ng_per_m3_precision,longitude,latitude,monitoring_equipment,time_resolution,instrument_type,instrument_name,station_name,station_id,time_resolution_timedelta,start_time,end_time,temperature
0,2020-01-08 13:45:00,623.0,NaN,49.0,2.964886,45.772223,ads_tube,5d,ads_tube,ads_tube_TERA-Env_unspecified,Puy de Dôme,FR0030R,5 days 00:00:00,2020-01-08 13:45:00,2020-01-13 13:45:00,2.242975
1,2020-01-23 13:31:00,638.0,NaN,52.0,2.964886,45.772223,ads_tube,5d,ads_tube,ads_tube_TERA-Env_unspecified,Puy de Dôme,FR0030R,5 days 00:00:00,2020-01-23 13:31:00,2020-01-28 13:31:00,1.166942
2,2020-01-31 13:53:00,714.0,NaN,58.0,2.964886,45.772223,ads_tube,5d,ads_tube,ads_tube_TERA-Env_unspecified,Puy de Dôme,FR0030R,5 days 00:00:00,2020-01-31 13:53:00,2020-02-05 13:53:00,3.084298
3,2020-02-20 13:48:00,602.0,NaN,49.0,2.964886,45.772223,ads_tube,5d,ads_tube,ads_tube_TERA-Env_unspecified,Puy de Dôme,FR0030R,5 days 00:00:00,2020-02-20 13:48:00,2020-02-25 13:48:00,4.422477
4,2020-02-27 13:46:00,432.0,NaN,33.0,2.964886,45.772223,ads_tube,5d,ads_tube,ads_tube_TERA-Env_unspecified,Puy de Dôme,FR0030R,5 days 00:00:00,2020-02-27 13:46:00,2020-03-03 13:46:00,-0.209091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92008,2023-12-31 18:44:00,70.0,NaN,7.1,8.530419,47.377586,online_gc,1h,online_gc,Agilent_MARKES-GC-FID_cn24502278_ZUE,Zürich-Kaserne,CH0010U,0 days 01:00:00,2023-12-31 18:44:00,2023-12-31 19:44:00,4.100000
92009,2023-12-31 19:53:00,124.0,NaN,7.1,8.530419,47.377586,online_gc,1h,online_gc,Agilent_MARKES-GC-FID_cn24502278_ZUE,Zürich-Kaserne,CH0010U,0 days 01:00:00,2023-12-31 19:53:00,2023-12-31 20:53:00,4.000000
92010,2023-12-31 21:02:00,128.0,NaN,7.1,8.530419,47.377586,online_gc,1h,online_gc,Agilent_MARKES-GC-FID_cn24502278_ZUE,Zürich-Kaserne,CH0010U,0 days 01:00:00,2023-12-31 21:02:00,2023-12-31 22:02:00,3.500000
92011,2023-12-31 22:11:00,64.0,NaN,7.1,8.530419,47.377586,online_gc,1h,online_gc,Agilent_MARKES-GC-FID_cn24502278_ZUE,Zürich-Kaserne,CH0010U,0 days 01:00:00,2023-12-31 22:11:00,2023-12-31 23:11:00,3.600000


In [11]:
def fetch_nearest_location(client, lat, lon, radius=10000):
    """
    获取给定坐标附近最近的一个测量站点及其传感器ID。

    Args:
        client (OpenAQ): OpenAQ 客户端实例。
        lat (float): 纬度。
        lon (float): 经度。
        radius (int): 搜索半径（米）。

    Returns:
        tuple or None: 最近站点的 location_id 和 sensors_id，若未找到则返回 None。
    """
    try:
        response = client.locations.list(
            coordinates=(lat, lon),
            radius=radius,
            limit=1  # 只获取最近的一个站点
        )
        print(f"Locations.list() response: {response}", flush=True)  # 调试信息
    except Exception as e:
        print(f"Error fetching locations: {e}", flush=True)
        return None

    locations = response.results
    if not locations:
        print(f"No nearby locations found for lat: {lat}, lon: {lon}", flush=True)
        return None

    nearest_location = locations[0]
    # 确认 'sensors' 属性存在且包含传感器信息
    if hasattr(nearest_location, 'sensors') and nearest_location.sensors:
        sensors = nearest_location.sensors
        # 假设每个传感器有一个 'id' 属性
        sensors_id = sensors[0].id if hasattr(sensors[0], 'id') else None
        location_id = nearest_location.name  # 或使用其他唯一标识符，如 'id'
        if sensors_id:
            print(f"Nearest location: {location_id}, Sensor ID: {sensors_id}", flush=True)
            return location_id, sensors_id
        else:
            print("Nearest location's sensor does not have a valid 'id' attribute.", flush=True)
            return None
    else:
        print("Nearest location does not have valid 'sensors' information.", flush=True)
        return None

def fetch_measurements(client, sensors_id, start_date, end_date, limit=1000):
    """
    获取指定传感器和时间范围的所有污染物测量数据。

    Args:
        client (OpenAQ): OpenAQ 客户端实例。
        sensors_id (int): 传感器的 ID。
        start_date (pd.Timestamp): 数据起始时间。
        end_date (pd.Timestamp): 数据结束时间。
        limit (int): 最大返回条数。

    Returns:
        pd.DataFrame: 包含所有污染物按小时重采样并计算平均值的数据。
    """
    try:
        response = client.measurements.list(
            sensors_id=sensors_id,  # 使用传感器 ID
            data='hours',           # 根据需要选择 'measurements', 'hours', etc.
            datetime_from=start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            datetime_to=end_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            limit=limit
        )
        print(f"Measurements.list() response: {response}", flush=True)  # 调试信息
    except Exception as e:
        print(f"Error fetching measurements: {e}", flush=True)
        return pd.DataFrame(columns=pollutants)

    if not response.results:
        print(f"No data found for sensors ID '{sensors_id}'", flush=True)
        return pd.DataFrame(columns=pollutants)

    # 处理 Measurement 对象，提取必要的信息
    measurements_list = []
    for measurement in response.results:
        # 提取 UTC 时间
        datetime_utc = None
        if hasattr(measurement.period, 'datetime_from') and hasattr(measurement.period.datetime_from, 'utc'):
            datetime_utc = measurement.period.datetime_from.utc
        elif 'datetime_from' in measurement.period.__dict__:
            datetime_utc = measurement.period.datetime_from.utc
        # 提取参数名称
        parameter_name = measurement.parameter.name if hasattr(measurement.parameter, 'name') else None

        if datetime_utc and parameter_name in pollutants:
            measurements_list.append({
                'datetime': datetime_utc,
                'parameter': parameter_name,
                'value': measurement.value
            })

    # 将处理后的数据转换为 DataFrame
    data_df = pd.DataFrame(measurements_list)
    if data_df.empty:
        print(f"DataFrame is empty for sensors ID '{sensors_id}'", flush=True)
        return pd.DataFrame(columns=pollutants)

    # 解析日期时间
    data_df['datetime'] = pd.to_datetime(data_df['datetime'], errors='coerce')
    data_df.dropna(subset=['datetime'], inplace=True)  # 移除无法解析的日期时间
    data_df.set_index('datetime', inplace=True)

    # 透视表，将每个污染物作为单独的列
    hourly_data = data_df.pivot_table(index='datetime', columns='parameter', values='value', aggfunc='mean')

    # 确保所有污染物都有列，即使某些污染物没有数据
    for pollutant in pollutants:
        if pollutant not in hourly_data.columns:
            hourly_data[pollutant] = np.nan

    # 按小时重采样并计算平均值
    hourly_data = hourly_data.resample('H').mean()

    print(f"Hourly data for pollutants:\n{hourly_data.head()}", flush=True)  # 调试信息
    return hourly_data

def fetch_nearby_pollutant_data(client, lat, lon, start_date, end_date, pollutants, radius=1000, delta_days=15):
    """
    获取给定坐标附近最近的一个测量站点的所有指定污染物数据，分15天一个间隔以避免API限制。

    Args:
        client (OpenAQ): OpenAQ 客户端实例。
        lat (float): 纬度。
        lon (float): 经度。
        start_date (pd.Timestamp): 数据起始时间。
        end_date (pd.Timestamp): 数据结束时间。
        pollutants (list): 污染物参数列表。
        radius (int): 半径（米）。
        delta_days (int): 每个时间间隔的天数。

    Returns:
        pd.DataFrame: 包含所有污染物按小时重采样并计算平均值的数据。
    """
    location_sensor = fetch_nearest_location(client, lat, lon, radius)
    if not location_sensor:
        print(f"No location sensor found for pollutants: {pollutants}", flush=True)
        return pd.DataFrame(columns=pollutants)

    location_id, sensors_id = location_sensor

    # 初始化一个空的 DataFrame 来存储所有数据
    all_data = pd.DataFrame()

    # 计算总的天数
    total_days = (end_date - start_date).days
    num_chunks = (total_days // delta_days) + 1
    print(f"共有{num_chunks}次")

    for i in range(num_chunks):
        chunk_start = start_date + pd.Timedelta(days=i * delta_days)
        chunk_end = min(chunk_start + pd.Timedelta(days=delta_days), end_date)
        print(f"Fetching data from {chunk_start} to {chunk_end}", flush=True)

        # 获取污染物数据
        chunk_data = fetch_measurements(client, sensors_id, chunk_start, chunk_end, limit=1000)
        if not chunk_data.empty:
            all_data = pd.concat([all_data, chunk_data])
            print(f"Appended data from {chunk_start} to {chunk_end}", flush=True)
        else:
            print(f"No data returned for chunk {i+1}: {chunk_start} to {chunk_end}", flush=True)

        # 为避免触发速率限制，添加延时
        print("Sleeping for 2 seconds to avoid rate limiting.", flush=True)
        time.sleep(2)

    if all_data.empty:
        print(f"No data found for pollutants {pollutants} at sensors ID '{sensors_id}'", flush=True)
        return pd.DataFrame(columns=pollutants)

    # 确保所有污染物都有列，即使某些污染物在部分时间段没有数据
    for pollutant in pollutants:
        if pollutant not in all_data.columns:
            all_data[pollutant] = np.nan

    # 按小时重采样并计算平均值
    all_data = all_data.resample('H').mean()

    print(f"Combined hourly data for pollutants:\n{all_data.head()}", flush=True)
    return all_data

# 主循环处理 matched_data 中的站点
print("Starting main loop to fetch data for each station.", flush=True)
for station_id, group in tqdm(matched_data.groupby('station_id'), desc='Fetching nearby station data'):
    print(f"\nProcessing station_id: {station_id}", flush=True)
    lat, lon = group['latitude'].iloc[0], group['longitude'].iloc[0]
    start_date, end_date = group['start_time'].min(), group['end_time'].max()
    print(f"Latitude: {lat}, Longitude: {lon}, Start: {start_date}, End: {end_date}", flush=True)

    # 初始化一个空的 DataFrame，索引为小时级别
    station_pollutants = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='H'))
    print("Initialized station_pollutants DataFrame with hourly index.", flush=True)

    # 获取所有污染物的数据，分15天一个间隔
    pollutant_data = fetch_nearby_pollutant_data(
        client, lat, lon, start_date, end_date, pollutants, radius=1000, delta_days=15
    )
    if not pollutant_data.empty:
        print(f"Joining data for all pollutants: {pollutants}", flush=True)
        try:
            # 使用 pd.concat 代替 join，并确保列对齐
            station_pollutants = pd.concat([station_pollutants, pollutant_data], axis=1)
            print(f"Joined data for all pollutants:\n{station_pollutants.head()}", flush=True)
        except Exception as e:
            print(f"Error joining data for pollutants {pollutants}: {e}", flush=True)
            for pollutant in pollutants:
                station_pollutants[pollutant] = np.nan
    else:
        print(f"No data returned for pollutants: {pollutants}", flush=True)
        for pollutant in pollutants:
            station_pollutants[pollutant] = np.nan

    print("Completed fetching all pollutants data.", flush=True)

    # 按照时间分辨率进行数据插值和平均匹配
    for idx, row in group.iterrows():
        res = row['time_resolution_timedelta']
        st, et = row['start_time'], row['end_time']
        print(f"\nProcessing row {idx}, resolution {res}, start_time {st}, end_time {et}", flush=True)

        try:
            if res < pd.Timedelta(hours=1):
                # 小于1小时的分辨率：插值至分钟级别
                print("  Interpolating data to 1-minute frequency.", flush=True)
                interp_pollutants = station_pollutants.reindex(
                    pd.date_range(start=station_pollutants.index.min(), end=station_pollutants.index.max(), freq='1min')
                ).interpolate(method='time')
                mask = (interp_pollutants.index >= st) & (interp_pollutants.index <= et)
                avg_pollutants = interp_pollutants.loc[mask].mean()
                print(f"  Average pollutants after interpolation:\n{avg_pollutants}", flush=True)
            else:
                # 等于或大于1小时：直接平均
                print("  Calculating direct average.", flush=True)
                mask = (station_pollutants.index >= st) & (station_pollutants.index <= et)
                avg_pollutants = station_pollutants.loc[mask].mean()
                print(f"  Average pollutants:\n{avg_pollutants}", flush=True)

            for pollutant in pollutants:
                value = avg_pollutants.get(pollutant, np.nan)
                matched_data.at[idx, pollutant] = value
                print(f"  Set pollutant {pollutant} to {value} for row {idx}", flush=True)
        except Exception as e:
            print(f"  Error processing row {idx}: {e}", flush=True)

    print(f"Completed processing station {station_id}", flush=True)


Starting main loop to fetch data for each station.


Fetching nearby station data:   0%|          | 0/5 [00:00<?, ?it/s]


Processing station_id: CH0010U
Latitude: 47.377586, Longitude: 8.530419, Start: 2023-01-01 00:10:00, End: 2024-01-01 00:20:00
Initialized station_pollutants DataFrame with hourly index.


Locations.list() response: LocationsResponse(headers=Headers(x_ratelimit_limit=60, x_ratelimit_remaining=1, x_ratelimit_used=59, x_ratelimit_reset=-2), meta=Meta(name='openaq-api', website='/', page=1, limit=1, found='>1'), results=[Location(id=5737, name='Zürich-Schimmelstrasse', locality='OSTLUFT', timezone='Europe/Zurich', country=CountryBase(id=92, code='CH', name='Switzerland'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provider=ProviderBase(id=70, name='EEA'), is_mobile=False, is_monitor=True, instruments=[InstrumentBase(id=2, name='Government Monitor')], sensors=[SensorBase(id=15467, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')), SensorBase(id=15465, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')), SensorBase(id=15463, name='pm10 µg/m³', parameter=ParameterBase(id=1, name='pm10', units='µg/m³', display_name='PM10')), SensorBase(id=8622699, name='so2 µg/

Fetching data from 2023-01-16 00:10:00 to 2023-01-31 00:10:00
Measurements.list() response: MeasurementsResponse(headers=Headers(x_ratelimit_limit=60, x_ratelimit_remaining=1, x_ratelimit_used=59, x_ratelimit_reset=-2), meta=Meta(name='openaq-api', website='/', page=1, limit=1000, found=127), results=[Measurement(period=Period(label='1hour', interval='01:00:00', datetime_from=Datetime(utc='2023-01-16T21:00:00Z', local='2023-01-16T22:00:00+01:00'), datetime_to=Datetime(utc='2023-01-16T22:00:00Z', local='2023-01-16T23:00:00+01:00')), value=5.7, parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name=None), coordinates=None, summary=Summary(min=5.6594, q02=5.6594, q25=5.6594, median=5.6594, q75=5.6594, q98=5.6594, max=5.6594, sd=None), coverage=Coverage(expected_count=1, expected_interval='01:00:00', observed_count=1, observed_interval='01:00:00', percent_complete=100.0, percent_coverage=100.0, datetime_from=Datetime(utc='2023-01-16T22:00:00Z', local='2023-01-16T23:00:00+01:

Fetching data from 2023-01-31 00:10:00 to 2023-02-15 00:10:00


Fetching nearby station data:   0%|          | 0/5 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [1]:
def fetch_nearest_location(client, lat, lon, radius=10000):
    """
    获取给定坐标附近最近的一个测量站点及其传感器ID。
    """
    try:
        response = client.locations.list(
            coordinates=(lat, lon),
            radius=radius,
            limit=1  # 只获取最近的一个站点
        )
        logging.info(f"Locations.list() response: {response}")  # 调试信息
    except Exception as e:
        logging.error(f"Error fetching locations: {e}")
        return None

    locations = response.results
    if not locations:
        logging.warning(f"No nearby locations found for lat: {lat}, lon: {lon}")
        return None

    nearest_location = locations[0]
    # 确认 'sensors' 属性存在且包含传感器信息
    if hasattr(nearest_location, 'sensors') and nearest_location.sensors:
        sensors = nearest_location.sensors
        # 假设每个传感器有一个 'id' 属性
        sensors_id = sensors[0].id if hasattr(sensors[0], 'id') else None
        location_id = nearest_location.name  # 或使用其他唯一标识符，如 'id'
        if sensors_id:
            logging.info(f"Nearest location: {location_id}, Sensor ID: {sensors_id}")
            return location_id, sensors_id
        else:
            logging.warning("Nearest location's sensor does not have a valid 'id' attribute.")
            return None
    else:
        logging.warning("Nearest location does not have valid 'sensors' information.")
        return None

def fetch_measurements(client, sensors_id, start_date, end_date, limit=1000, max_retries=3):
    """
    获取指定传感器和时间范围的所有污染物测量数据，处理HTTP 500错误并进行重试。
    """
    attempt = 0
    backoff = 1  # 初始回退时间（秒）

    while attempt < max_retries:
        try:
            response = client.measurements.list(
                sensors_id=sensors_id,  # 使用传感器 ID
                data='hours',           # 根据需要选择 'measurements', 'hours', etc.
                datetime_from=start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
                datetime_to=end_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
                limit=limit
            )
            logging.info(f"Measurements.list() response: {response}")  # 调试信息
        except Exception as e:
            logging.error(f"Error fetching measurements: {e}")
            return pd.DataFrame(columns=pollutants)

        if not response.results:
            logging.warning(f"No data found for sensors ID '{sensors_id}'")
            return pd.DataFrame(columns=pollutants)

        # 处理 Measurement 对象，提取必要的信息
        measurements_list = []
        for measurement in response.results:
            # 提取 UTC 时间
            datetime_utc = None
            if hasattr(measurement.period, 'datetime_from') and hasattr(measurement.period.datetime_from, 'utc'):
                datetime_utc = measurement.period.datetime_from.utc
            elif 'datetime_from' in measurement.period.__dict__:
                datetime_utc = measurement.period.datetime_from.utc
            # 提取参数名称
            parameter_name = measurement.parameter.name if hasattr(measurement.parameter, 'name') else None

            if datetime_utc and parameter_name in pollutants:
                measurements_list.append({
                    'datetime': datetime_utc,
                    'parameter': parameter_name,
                    'value': measurement.value
                })

        # 将处理后的数据转换为 DataFrame
        data_df = pd.DataFrame(measurements_list)
        if data_df.empty:
            logging.warning(f"DataFrame is empty for sensors ID '{sensors_id}'")
            return pd.DataFrame(columns=pollutants)

        # 解析日期时间
        data_df['datetime'] = pd.to_datetime(data_df['datetime'], errors='coerce')
        data_df.dropna(subset=['datetime'], inplace=True)  # 移除无法解析的日期时间
        data_df.set_index('datetime', inplace=True)

        # 透视表，将每个污染物作为单独的列
        hourly_data = data_df.pivot_table(index='datetime', columns='parameter', values='value', aggfunc='mean')

        # 确保所有污染物都有列，即使某些污染物没有数据
        for pollutant in pollutants:
            if pollutant not in hourly_data.columns:
                hourly_data[pollutant] = np.nan

        # 按小时重采样并计算平均值
        hourly_data = hourly_data.resample('h').mean()  # 修复FutureWarning，将'H'替换为'h'

        logging.info(f"Hourly data for pollutants:\n{hourly_data.head()}")
        return hourly_data

    # 如果所有重试都失败
    logging.error(f"Failed to fetch measurements after {max_retries} attempts.")
    return pd.DataFrame(columns=pollutants)

def fetch_nearby_pollutant_data(client, lat, lon, start_date, end_date, pollutants, radius=1000, delta_days=15):
    """
    获取给定坐标附近最近的一个测量站点的所有指定污染物数据，分15天一个间隔以避免API限制。
    """
    location_sensor = fetch_nearest_location(client, lat, lon, radius)
    if not location_sensor:
        logging.warning(f"No location sensor found for pollutants: {pollutants}")
        return pd.DataFrame(columns=pollutants)

    location_id, sensors_id = location_sensor

    # 初始化一个空的 DataFrame 来存储所有数据
    all_data = pd.DataFrame()

    # 计算总的天数
    total_days = (end_date - start_date).days
    num_chunks = (total_days // delta_days) + 1

    for i in range(num_chunks):
        chunk_start = start_date + pd.Timedelta(days=i * delta_days)
        chunk_end = min(chunk_start + pd.Timedelta(days=delta_days), end_date)
        logging.info(f"Fetching data from {chunk_start} to {chunk_end}")

        # 获取污染物数据
        chunk_data = fetch_measurements(client, sensors_id, chunk_start, chunk_end, limit=1000)
        if not chunk_data.empty:
            all_data = pd.concat([all_data, chunk_data])
            logging.info(f"Appended data from {chunk_start} to {chunk_end}")
        else:
            logging.info(f"No data returned for chunk {i+1}: {chunk_start} to {chunk_end}")

        # 为避免触发速率限制，添加延时
        logging.info("Sleeping for 2 seconds to avoid rate limiting.")
        time.sleep(2)

    if all_data.empty:
        logging.warning(f"No data found for pollutants {pollutants} at sensors ID '{sensors_id}'")
        return pd.DataFrame(columns=pollutants)

    # 确保所有污染物都有列，即使某些污染物在部分时间段没有数据
    for pollutant in pollutants:
        if pollutant not in all_data.columns:
            all_data[pollutant] = np.nan

    # 按小时重采样并计算平均值
    all_data = all_data.resample('h').mean()  # 修复FutureWarning，将'H'替换为'h'

    logging.info(f"Combined hourly data for pollutants:\n{all_data.head()}")
    return all_data

# 主循环处理 matched_data 中的站点
logging.info("Starting main loop to fetch data for each station.")
for station_id, group in tqdm(matched_data.groupby('station_id'), desc='Fetching nearby station data'):
    logging.info(f"\nProcessing station_id: {station_id}")
    lat, lon = group['latitude'].iloc[0], group['longitude'].iloc[0]
    start_date, end_date = group['start_time'].min(), group['end_time'].max()
    logging.info(f"Latitude: {lat}, Longitude: {lon}, Start: {start_date}, End: {end_date}")

    # 初始化一个空的 DataFrame，索引为小时级别
    station_pollutants = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='h'))  # 修复FutureWarning，将'H'替换为'h'
    logging.info("Initialized station_pollutants DataFrame with hourly index.")

    # 获取所有污染物的数据，分15天一个间隔
    pollutant_data = fetch_nearby_pollutant_data(
        client, lat, lon, start_date, end_date, pollutants, radius=1000, delta_days=15
    )
    if not pollutant_data.empty:
        logging.info(f"Joining data for all pollutants: {pollutants}")
        try:
            # 使用 pd.concat 代替 join，并确保列对齐
            station_pollutants = pd.concat([station_pollutants, pollutant_data], axis=1)
            logging.info(f"Joined data for all pollutants:\n{station_pollutants.head()}")
        except Exception as e:
            logging.error(f"Error joining data for pollutants {pollutants}: {e}")
            for pollutant in pollutants:
                station_pollutants[pollutant] = np.nan
    else:
        logging.info(f"No data returned for pollutants: {pollutants}")
        for pollutant in pollutants:
            station_pollutants[pollutant] = np.nan

    logging.info("Completed fetching all pollutants data.")

    # 按照时间分辨率进行数据插值和平均匹配
    for idx, row in group.iterrows():
        res = row['time_resolution_timedelta']
        st, et = row['start_time'], row['end_time']
        logging.info(f"\nProcessing row {idx}, resolution {res}, start_time {st}, end_time {et}")

        try:
            if res < pd.Timedelta(hours=1):
                # 小于1小时的分辨率：插值至分钟级别
                logging.info("  Interpolating data to 1-minute frequency.")
                interp_pollutants = station_pollutants.reindex(
                    pd.date_range(start=station_pollutants.index.min(), end=station_pollutants.index.max(), freq='1min')
                ).interpolate(method='time')
                mask = (interp_pollutants.index >= st) & (interp_pollutants.index <= et)
                avg_pollutants = interp_pollutants.loc[mask].mean()
                logging.info(f"  Average pollutants after interpolation:\n{avg_pollutants}")
            else:
                # 等于或大于1小时：直接平均
                logging.info("  Calculating direct average.")
                mask = (station_pollutants.index >= st) & (station_pollutants.index <= et)
                avg_pollutants = station_pollutants.loc[mask].mean()
                logging.info(f"  Average pollutants:\n{avg_pollutants}")

            for pollutant in pollutants:
                value = avg_pollutants.get(pollutant, np.nan)
                matched_data.at[idx, pollutant] = value
                logging.info(f"  Set pollutant {pollutant} to {value} for row {idx}")
        except Exception as e:
            logging.error(f"  Error processing row {idx}: {e}")

    logging.info(f"Completed processing station {station_id}")

NameError: name 'logging' is not defined